<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/notebooks/rnn_gradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explaining Exploding and Vanishing Gradients

- From [The Vanishing Gradient Problem](https://towardsdatascience.com/the-vanishing-gradient-problem-69bf08b15484) by [Chi-Feng Wang](https://towardsdatascience.com/@reina.wang) in [towardsdatascience.com](https://towardsdatascience.com)
- and From [The curious case of the vanishing & exploding gradient](https://medium.com/learn-love-ai/the-curious-case-of-the-vanishing-exploding-gradient-bf58ec6822eb) by [Eniola Alese](https://medium.com/@eniola.alese?source=post_page-----bf58ec6822eb----------------------) in [www.medium.com](https://www.medium.com/)

Updated by [John Fogarty](https://github.com/jfogarty) for Python 3.6 and [Base2 MLI](https://github.com/base2solutions/mli).


## The problem of Vanishing Gradients:

As more layers using certain activation functions are added to neural networks, the gradients of the loss function approaches zero, making the network hard to train.

Why:
Certain activation functions, like the sigmoid function, squishes a large input space into a small input space between $0$ and $1$. Therefore, a large change in the input of the sigmoid function will cause a small change in the output. Hence, the derivative becomes small.

<figure>
  <br><center>
    <img src="../images/vg_1.png" />
    <figcaption>Image 1: The sigmoid function and its derivative // <a src="Image 1: The sigmoid function and its derivative // Source">Source</a></figcaption>     
  </center>
</figure>


As an example, Image 1 is the sigmoid function and its derivative. Note how when the inputs of the sigmoid function becomes larger or smaller (when $|x|$ becomes bigger), the derivative becomes close to zero.

## Why it’s significant:

For shallow network with only a few layers that use these activations, this isn’t a big problem. However, when more layers are used, it can cause the gradient to be too small for training to work effectively.

Gradients of neural networks are found using backpropagation. Simply put, backpropagation finds the derivatives of the network by moving layer by layer from the final layer to the initial one. By the chain rule, the derivatives of each layer are multiplied down the network (from the final layer to the initial) to compute the derivatives of the initial layers.

**However, when n hidden layers use an activation like the sigmoid function, n small derivatives are multiplied together. Thus, the gradient decreases exponentially as we propagate down to the initial layers.**

A small gradient means that the weights and biases of the initial layers will not be updated effectively with each training session. Since these initial layers are often crucial to recognizing the core elements of the input data, it can lead to overall inaccuracy of the whole network.

## Solutions:

The simplest solution is to use other activation functions, such as ReLU, which doesn’t cause a small derivative.

[Residual networks]() are another solution, as they provide residual connections straight to earlier layers. As seen in *Image 2*, the residual connection directly adds the value at the beginning of the block, x, to the end of the block (F(x)+x). This residual connection doesn’t go through activation functions that “squashes” the derivatives, resulting in a higher overall derivative of the block.

<figure>
  <br><center>
    <img src="../images/vg_2.png" />
    <figcaption>Image 1: A residual block</figcaption>     
  </center>
</figure>

Finally, batch normalization layers can also resolve the issue. As stated before, the problem arises when a large input space is mapped to a small one, causing the derivatives to disappear. In Image 1, this is most clearly seen at when $|x|$ is big. Batch normalization reduces this problem by simply normalizing the input so $|x|$ doesn’t reach the outer edges of the sigmoid function. As seen in *Image 3*, it normalizes the input so that most of it falls in the green region, where the derivative isn’t too small.

<figure>
  <br><center>
    <img src="../images/vg_3.png" />
    <figcaption>Image 3: Sigmoid function with restricted inputs</figcaption>     
  </center>
</figure>

# Gradient Problems in Sequential Networks

In [a previous post](https://medium.com/learn-love-ai/step-by-step-walkthrough-of-rnn-training-part-ii-7141084d274b), we introduced a step by step walkthrough of RNN training and how to derive the gradients of the network weights using back propagation and the chain rule. But it turns out that during this training the RNN can suffer greatly from two problems: 1. Vanishing gradients or 2. Exploding gradients.

## Why Gradients Explode or Vanish in RNNs

Recall the many-to-many architecture for text generation shown below and in the introduction to RNN post, lets assume the input sequence to the network is a 20 word sentence: *“I grew up in France,…….. I speak French fluently"*.

<figure>
  <br><center>
    <img src="../images/rex_1.png" />
    <figcaption></figcaption>     
  </center>
</figure>

We can see from the example above that for the RNN to predict the word “French” which comes at the end of the sequence, it would need information from the word ***“France”***, which occurs further back at the beginning of the sentence. This kind of dependence between sequence data is called long-term dependencies because the distance between the relevant information ***“France”*** and the point where it is needed to make a prediction ***“French”*** is very wide. Unfortunately, in practice as this distance becomes wider, RNNs have a hard time learning these dependencies because it encounters either a vanishing or exploding gradient problem.

These problems arise during training of a deep network when the gradients are being propagated back in time all the way to the initial layer. The gradients coming from the deeper layers have to go through continuous matrix multiplications because of the the chain rule, and as they approach the earlier layers, if they have small values ($<1$), they shrink exponentially until they vanish and make it impossible for the model to learn , this is the *vanishing gradient problem*. While on the other hand if they have large values ($>1$) they get larger and eventually blow up and crash the model, this is the *exploding gradient problem*

## Dealing with Exploding Gradients

When gradients explode, the gradients could become `NaN` because of the numerical overflow or we might see irregular oscillations in training cost when we plot the learning curve. A solution to fix this is to apply **[gradient clipping](http://proceedings.mlr.press/v28/pascanu13.pdf)**; which places a predefined threshold on the gradients to prevent it from getting too large, and by doing this it doesn’t change the direction of the gradients it only change its length.


<figure>
  <br><center>
    <img src="../images/rex_2.png" />
    <figcaption>Reference: Ian Goodfellow et. al, “<a src="http://www.deeplearningbook.org/contents/rnn.html">Deep Learning</a>”, MIT press, 2016</figcaption>     
  </center>
</figure>

## Dealing with Vanishing Gradients

One simple solution for dealing with vanishing gradient is the **[identity RNN](https://arxiv.org/pdf/1504.00941.pdf)** architecture; where the network weights are initialized to the identity matrix and the activation functions are all set to ReLU and this ends up encouraging the network computations to stay close to the identity function. This works well because when the error derivatives are being propagated backwards through time, they remain constants of either 0 or 1, hence aren’t likely to suffer from vanishing gradients.

An even more popular and widely used solution is the **[Long Short-Term Memory](http://www.bioinf.jku.at/publications/older/2604.pdf)** architecture (LSTM); a variant of the regular recurrent network which was designed to make it easy to capture long-term dependencies in sequence data. The standard RNN operates in such a way that the hidden state activation are influenced by the other local activations closest to them, which corresponds to a *“short-term memory”*, while the network weights are influenced by the computations that take place over entire long sequences, which corresponds to a *“long-term memory”*. Hence the RNN was redesigned so that it has an activation state that can also act like weights and preserve information over long distances, hence the name *“Long Short-Term Memory”*.


<figure>
  <br><center>
    <img src="../images/rex_3.gif" />
    <figcaption>LSTM Unit</figcaption>     
  </center>
</figure>

## Conclusion

In this post we explored the reasons why gradients explode or vanish and examined some methods for dealing with the problem. In the next post we would look further into the LSTM architecture and learn how to derive its gradients for back propagation.

### End of notebook